<a href="https://colab.research.google.com/github/JiwonOcho/ML-DL/blob/main/NLP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
5_5_seq2seq_csv파일로부터_로드.ipynb

Text cell <1ef8p19lsTmb>
#%% [markdown]
[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/tensorflow/blob/main/Part%205/5.5_seq2seq.ipynb)

Text cell <pV5qtxxcnKLg>
#%% [markdown]
## Part5-4-4 시퀀스 투 시퀀스 

Code cell <SQrA1dJQbgnZ>
#%% [code]
import pandas as pd

corpus = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')
corpus.head()
Execution output from Aug 10, 2022 7:10 AM
5KB
	text/plain
		Q            A  label
		0           12시 땡!   하루가 또 가네요.      0
		1      1지망 학교 떨어졌어    위로해 드립니다.      0
		2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
		3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
		4          PPL 심하네   눈살이 찌푸려지죠.      0

Code cell <aJCsCjLknKLj>
#%% [code]
# 챗봇 데이터 샘플 확인
display(corpus['Q'].head())
print('==='*10)
display(corpus['A'].head())
Execution output from Aug 10, 2022 7:12 AM
0KB
	Stream
		==============================
	text/plain
		0             12시 땡!
		1        1지망 학교 떨어졌어
		2       3박4일 놀러가고 싶다
		3    3박4일 정도 놀러가고 싶다
		4            PPL 심하네
		Name: Q, dtype: object
		0     하루가 또 가네요.
		1      위로해 드립니다.
		2    여행은 언제나 좋죠.
		3    여행은 언제나 좋죠.
		4     눈살이 찌푸려지죠.
		Name: A, dtype: object

Code cell <us69ER49nKLk>
#%% [code]
# text와 pair가 쌍으로 이루어짐
print("Q:", corpus['Q'][0])
print("A:", corpus['A'][0])
Execution output from Aug 10, 2022 7:12 AM
0KB
	Stream
		Q: 12시 땡!
		A: 하루가 또 가네요.

Text cell <_19hU_nhnKLk>
#%% [markdown]
## 데이터 전처리

Code cell <iwYDPUUpRHgc>
#%% [code]
# 전체 코퍼스 크기 (11,823개)
corpus.shape[0]
Execution output from Aug 10, 2022 7:12 AM
0KB
	text/plain
		11823

Code cell <yl0H2-RsnKLk>
#%% [code]
# 2,000개 데이터 셋만 활용 (Google Colab 일 경우 3,000개에서는 메모리 오버되는 현상 발생)
texts = []
pairs = []
for i, (text, pair) in enumerate(zip(corpus['Q'], corpus['A'])):
    texts.append(text)
    pairs.append(pair)
    if i >= 2000: 
        break 

Code cell <uWuUWju3nKLl>
#%% [code]
# question와 answer 데이터 확인
list(zip(texts, pairs))[1995:2000]
Execution output from Aug 10, 2022 7:12 AM
0KB
	text/plain
		[('배 아프다', '약이 필요하면 도움을 받아보세요.'),
		 ('배 터지겠네', '위를 좀 쉬게 해주세요.'),
		 ('배 터지겠다.', '산책 좀 해야겠네여.'),
		 ('배가 너무 고파', '뭐 좀 챙겨드세요.'),
		 ('배가 넘넘 고파', '저도 밥 먹고 싶어요')]

Text cell <lkp1mmttnKLl>
#%% [markdown]
### 특수문자는 제거합니다.

Code cell <mnQ5Xb58nKLl>
#%% [code]
# 데이터 전처리 함수 (정규식(regex expression)활용)
import re
def clean_sentence(sentence):
    # 한글, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

Code cell <ElslA3SgnKLl>
#%% [code]
# 전처리 함수 테스트
print(clean_sentence('안녕하세요~:)'))
print(clean_sentence('텐서플로^@^%#@!'))
Execution output from Aug 10, 2022 7:12 AM
0KB
	Stream
		안녕하세요
		텐서플로

Text cell <4TPsL7c-nKLm>
#%% [markdown]
### 한글 형태소 분석기 (Konlpy)

Code cell <7DnDlZwxnKLm>
#%% [code]
# konlpy 설치
!pip install konlpy
Execution output from Aug 10, 2022 7:10 AM
1KB
	Stream
		Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
		Collecting konlpy
		  Downloading konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
		[K     |████████████████████████████████| 19.4 MB 1.4 MB/s 
		[?25hRequirement already satisfied: numpy>=1.6 in /usr/local/lib/python3.7/dist-packages (from konlpy) (1.21.6)
		Collecting JPype1>=0.7.0
		  Downloading JPype1-1.4.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (453 kB)
		[K     |████████████████████████████████| 453 kB 43.1 MB/s 
		[?25hRequirement already satisfied: lxml>=4.1.0 in /usr/local/lib/python3.7/dist-packages (from konlpy) (4.9.1)
		Requirement already satisfied: typing-extensions in /usr/local/lib/python3.7/dist-packages (from JPype1>=0.7.0->konlpy) (4.1.1)
		Installing collected packages: JPype1, konlpy
		Successfully installed JPype1-1.4.0 konlpy-0.6.0

Code cell <Jeu4-LHHnKLn>
#%% [code]
# Okt형태소
from konlpy.tag import Okt
okt = Okt()
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

Code cell <lv0cn1SDnKLn>
#%% [code]
# 문장 전처리
def clean_and_morph(sentence, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)

    # 형태소 변환
    sentence = process_morph(sentence)
    
    # Question 인 경우, Answer인 경우를 분기하여 처리합니다.
    if is_question:
        return sentence
    else:
        # START 토큰은 decoder input에 END 토큰은 decoder output에 추가합니다.
        return ('<START> ' + sentence, sentence + ' <END>')

def preprocess(texts, pairs):
    questions = []
    answer_in = []
    answer_out = []

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, is_question=True)
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)
    
    return questions, answer_in, answer_out

questions, answer_in, answer_out = preprocess(texts, pairs)
print(questions[:2])
print(answer_in[:2])
print(answer_out[:2])
Execution output from Aug 10, 2022 7:12 AM
0KB
	Stream
		['12시 땡', '1 지망 학교 떨어졌어']
		['<START> 하루 가 또 가네요', '<START> 위로 해 드립니다']
		['하루 가 또 가네요 <END>', '위로 해 드립니다 <END>']

Code cell <sIweX12CnKLo>
#%% [code]
# 전체 문장을 하나의 리스트로 만들기
all_sentences = questions + answer_in + answer_out

Text cell <PRF7_GT9nKLo>
#%% [markdown]
## 토큰화

Code cell <CV5E3CZSnKLp>
#%% [code]
# 라이브러리 불어오기
import numpy as np
import warnings
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# WARNING 무시
warnings.filterwarnings('ignore')

Code cell <0vA-7xsFnKLp>
#%% [code]
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
tokenizer.fit_on_texts(all_sentences)

Code cell <g3FBSeF0nKLp>
#%% [code]
# 단어사전 확인
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t -> \t{idx}')
    if idx > 10:
        break
Execution output from Aug 10, 2022 7:12 AM
0KB
	Stream
		<OOV>	 -> 	1
		<START>	 -> 	2
		<END>	 -> 	3
		이	 -> 	4
		을	 -> 	5
		거	 -> 	6
		가	 -> 	7
		예요	 -> 	8
		도	 -> 	9
		해보세요	 -> 	10
		요	 -> 	11

Code cell <QCA-r61YnKLq>
#%% [code]
# 토큰 갯수 확인
len(tokenizer.word_index)
Execution output from Aug 10, 2022 7:12 AM
0KB
	text/plain
		3604

Text cell <L4VTDJxSnKLq>
#%% [markdown]
## 치환: 텍스트를 시퀀스로 인코딩 (`texts_to_sequences`)

Code cell <xM0c79UnnKLq>
#%% [code]
# 치환: 텍스트를 시퀀스로 인코딩 (texts_to_sequences)
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

# 문장의 길이 맞추기 (pad_sequences)
MAX_LENGTH = 30
question_padded = pad_sequences(question_sequence, 
                                maxlen=MAX_LENGTH, 
                                truncating='post', 
                                padding='post')
answer_in_padded = pad_sequences(answer_in_sequence, 
                                 maxlen=MAX_LENGTH, 
                                 truncating='post', 
                                 padding='post')
answer_out_padded = pad_sequences(answer_out_sequence, 
                                  maxlen=MAX_LENGTH, 
                                  truncating='post', 
                                  padding='post')

Code cell <PP6ujoVQnKLr>
#%% [code]
question_padded.shape, answer_in_padded.shape, answer_out_padded.shape
Execution output from Aug 10, 2022 7:13 AM
0KB
	text/plain
		((2001, 30), (2001, 30), (2001, 30))

Text cell <y2SKSCVTnKLr>
#%% [markdown]
## 모델

Code cell <CqsTUuxXnKLr>
#%% [code]
# 라이브러리 로드
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

Text cell <8zRAumSEnKLr>
#%% [markdown]
## 학습용 인코더 (Encoder)

Code cell <04jAUIC5nKLr>
#%% [code]
# 인코더
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True)
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        return [hidden_state, cell_state]

Text cell <UjhVwZNgnKLr>
#%% [markdown]
## 학습용 디코더 (Decoder)

Code cell <J4qC3Al0nKLs>
#%% [code]
# 디코더
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                        )
        self.dense = Dense(vocab_size, activation='softmax')
    
    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)        
        x = self.dense(x)
        return x, hidden_state, cell_state

Text cell <6CmUTWKknKLs>
#%% [markdown]
## 모델 결합

Code cell <huOfpcSQnKLs>
#%% [code]
# 모델 결합
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps
        
        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
        
    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            context_vector = self.encoder(encoder_inputs)
            decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, 
                                                 initial_state=context_vector)
            return decoder_outputs
        else:
            context_vector = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)
            
            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(target_seq, 
                                                                            initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), 
                                         dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)
                
                if decoder_output == self.end_token:
                    break
                    
                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]
                
            return tf.reshape(results.stack(), shape=(1, self.time_steps))

Text cell <RLR7e81DnKLt>
#%% [markdown]
## 단어별 원핫인코딩 적용

Code cell <gCNm7zCinKLt>
#%% [code]
VOCAB_SIZE = len(tokenizer.word_index)+1
VOCAB_SIZE
Execution output from Aug 10, 2022 7:13 AM
0KB
	text/plain
		3605

Code cell <IfWmtw_DnKLt>
#%% [code]
def convert_to_one_hot(padded):
    # 원핫인코딩 초기화
    one_hot_vector = np.zeros((len(answer_out_padded), 
                               MAX_LENGTH, 
                               VOCAB_SIZE))

    # 디코더 목표를 원핫인코딩으로 변환
    # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
    for i, sequence in enumerate(answer_out_padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1

    return one_hot_vector

answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)
answer_in_one_hot[0].shape, answer_in_one_hot[0].shape
Execution output from Aug 10, 2022 7:13 AM
0KB
	text/plain
		((30, 3605), (30, 3605))

Code cell <O-upnUy_nKLv>
#%% [code]
# 변환된 index를 다시 단어로 변환
def convert_index_to_text(indexs, end_token): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == end_token:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''
            
        # 빈칸 추가
        sentence += ' '
    return sentence

Text cell <sYvjxOEJnKLv>
#%% [markdown]
## 학습 (Training)

Code cell <_bcNTBJSnKLv>
#%% [code]
# 하이퍼 파라미터 정의
BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']

UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
NUM_EPOCHS = 20

Code cell <MHPtTUEmnKLv>
#%% [code]
# 체크포인트 생성
checkpoint_path = 'model/seq2seq-chatbot-no-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )

Code cell <CD0k5Yv-tL9z>
#%% [code]
# seq2seq
seq2seq = Seq2Seq(UNITS, 
                  VOCAB_SIZE, 
                  EMBEDDING_DIM, 
                  TIME_STEPS, 
                  START_TOKEN, 
                  END_TOKEN)

seq2seq.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['acc'])

Code cell <zfbT_Eo5nKLw>
#%% [code]
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

Code cell <D6pZYUJ6nKLw>
#%% [code]
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
        
        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)
        
        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()
Execution output from Aug 10, 2022 7:21 AM
62KB
	Stream
		processing epoch: 1...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 2.4240 - acc: 0.7934
		Epoch 1: loss improved from inf to 2.42348, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 10s 15ms/step - loss: 2.4235 - acc: 0.7934
		Epoch 2/10
		123/126 [============================>.] - ETA: 0s - loss: 1.2095 - acc: 0.8171
		Epoch 2: loss improved from 2.42348 to 1.20877, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 1.2088 - acc: 0.8174
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 1.1095 - acc: 0.8371
		Epoch 3: loss improved from 1.20877 to 1.10949, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 1.1095 - acc: 0.8371
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 1.0587 - acc: 0.8398
		Epoch 4: loss improved from 1.10949 to 1.05868, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 1.0587 - acc: 0.8398
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 1.0212 - acc: 0.8411
		Epoch 5: loss improved from 1.05868 to 1.02355, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 1.0236 - acc: 0.8408
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.9960 - acc: 0.8435
		Epoch 6: loss improved from 1.02355 to 0.99605, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.9961 - acc: 0.8435
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.9726 - acc: 0.8458
		Epoch 7: loss improved from 0.99605 to 0.97243, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.9724 - acc: 0.8458
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.9473 - acc: 0.8478
		Epoch 8: loss improved from 0.97243 to 0.95005, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.9501 - acc: 0.8475
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.9308 - acc: 0.8482
		Epoch 9: loss improved from 0.95005 to 0.93001, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.9300 - acc: 0.8483
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.9125 - acc: 0.8490
		Epoch 10: loss improved from 0.93001 to 0.91210, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.9121 - acc: 0.8491
		Q: 말투 가 맘 에 안 들어
		A: 저 을 더 해보세요 
		
		
		Q: 모르는 사람 이 자꾸 쳐다봐
		A: 저 을 있을 거 예요 
		
		
		Q: 그냥 혼자 밥 이나 먹어야지
		A: 저 을 더 해보세요 
		
		
		processing epoch: 11...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.8956 - acc: 0.8506
		Epoch 1: loss improved from 0.91210 to 0.89557, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.8956 - acc: 0.8506
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.8788 - acc: 0.8512
		Epoch 2: loss improved from 0.89557 to 0.87880, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.8788 - acc: 0.8512
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.8622 - acc: 0.8532
		Epoch 3: loss improved from 0.87880 to 0.86219, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.8622 - acc: 0.8532
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.8445 - acc: 0.8551
		Epoch 4: loss improved from 0.86219 to 0.84441, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.8444 - acc: 0.8552
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.8251 - acc: 0.8581
		Epoch 5: loss improved from 0.84441 to 0.82595, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.8260 - acc: 0.8579
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.8058 - acc: 0.8606
		Epoch 6: loss improved from 0.82595 to 0.80664, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.8066 - acc: 0.8606
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.7868 - acc: 0.8634
		Epoch 7: loss improved from 0.80664 to 0.78667, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.7867 - acc: 0.8634
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.7707 - acc: 0.8651
		Epoch 8: loss improved from 0.78667 to 0.77068, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.7707 - acc: 0.8651
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.7476 - acc: 0.8684
		Epoch 9: loss improved from 0.77068 to 0.74926, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.7493 - acc: 0.8681
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.7294 - acc: 0.8707
		Epoch 10: loss improved from 0.74926 to 0.72925, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.7293 - acc: 0.8707
		Q: 미용 몸무게 하고 싶다
		A: 저 도 요 
		
		
		Q: 나 는 그냥 저 냥 사는 거 같아
		A: 저 도 요 
		
		
		Q: 말문 이 탁 막힌다
		A: 저 도 요 
		
		
		processing epoch: 21...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.7080 - acc: 0.8735
		Epoch 1: loss improved from 0.72925 to 0.70882, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.7088 - acc: 0.8734
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.6901 - acc: 0.8755
		Epoch 2: loss improved from 0.70882 to 0.69023, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.6902 - acc: 0.8755
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.6715 - acc: 0.8781
		Epoch 3: loss improved from 0.69023 to 0.67150, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.6715 - acc: 0.8781
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.6520 - acc: 0.8808
		Epoch 4: loss improved from 0.67150 to 0.65270, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.6527 - acc: 0.8806
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.6342 - acc: 0.8831
		Epoch 5: loss improved from 0.65270 to 0.63423, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.6342 - acc: 0.8831
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.6166 - acc: 0.8853
		Epoch 6: loss improved from 0.63423 to 0.61664, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.6166 - acc: 0.8853
		Epoch 7/10
		123/126 [============================>.] - ETA: 0s - loss: 0.5996 - acc: 0.8878
		Epoch 7: loss improved from 0.61664 to 0.59954, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.5995 - acc: 0.8878
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.5806 - acc: 0.8905
		Epoch 8: loss improved from 0.59954 to 0.58154, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.5815 - acc: 0.8903
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.5638 - acc: 0.8937
		Epoch 9: loss improved from 0.58154 to 0.56578, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.5658 - acc: 0.8933
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.5495 - acc: 0.8968
		Epoch 10: loss improved from 0.56578 to 0.54935, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.5493 - acc: 0.8969
		Q: 대빵 큰 여드름 올라와
		A: 저 도 요 
		
		
		Q: 머리 가 띵해
		A: 저 도 요 
		
		
		Q: 내일 만나자고 해볼까
		A: 저 도 궁금하네요 
		
		
		processing epoch: 31...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.5333 - acc: 0.8997
		Epoch 1: loss improved from 0.54935 to 0.53326, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.5333 - acc: 0.8997
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.5192 - acc: 0.9022
		Epoch 2: loss improved from 0.53326 to 0.51867, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.5187 - acc: 0.9023
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.5042 - acc: 0.9052
		Epoch 3: loss improved from 0.51867 to 0.50499, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.5050 - acc: 0.9049
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.4899 - acc: 0.9081
		Epoch 4: loss improved from 0.50499 to 0.48992, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4899 - acc: 0.9081
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.4759 - acc: 0.9117
		Epoch 5: loss improved from 0.48992 to 0.47588, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4759 - acc: 0.9117
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.4627 - acc: 0.9140
		Epoch 6: loss improved from 0.47588 to 0.46319, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4632 - acc: 0.9138
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.4506 - acc: 0.9168
		Epoch 7: loss improved from 0.46319 to 0.45064, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4506 - acc: 0.9168
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.4383 - acc: 0.9189
		Epoch 8: loss improved from 0.45064 to 0.43843, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4384 - acc: 0.9189
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.4267 - acc: 0.9211
		Epoch 9: loss improved from 0.43843 to 0.42694, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4269 - acc: 0.9211
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.4149 - acc: 0.9242
		Epoch 10: loss improved from 0.42694 to 0.41535, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4154 - acc: 0.9240
		Q: 내 가 기대 를 너무 많이 했나 봐
		A: 저 도 궁금하네요 
		
		
		Q: 눈 쌓였어
		A: 잘 하고 있어요 
		
		
		Q: 너무 뻔 뻔하게 구 는 데
		A: 저 도 궁금하네요 
		
		
		processing epoch: 41...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.4101 - acc: 0.9249
		Epoch 1: loss improved from 0.41535 to 0.41013, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.4101 - acc: 0.9249
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.3964 - acc: 0.9278
		Epoch 2: loss improved from 0.41013 to 0.39647, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3965 - acc: 0.9278
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.3864 - acc: 0.9293
		Epoch 3: loss improved from 0.39647 to 0.38641, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3864 - acc: 0.9293
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.3768 - acc: 0.9309
		Epoch 4: loss improved from 0.38641 to 0.37702, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3770 - acc: 0.9308
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.3674 - acc: 0.9334
		Epoch 5: loss improved from 0.37702 to 0.36864, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3686 - acc: 0.9332
		Epoch 6/10
		126/126 [==============================] - ETA: 0s - loss: 0.3616 - acc: 0.9339
		Epoch 6: loss improved from 0.36864 to 0.36163, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3616 - acc: 0.9339
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.3532 - acc: 0.9355
		Epoch 7: loss improved from 0.36163 to 0.35321, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.3532 - acc: 0.9355
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.3457 - acc: 0.9371
		Epoch 8: loss improved from 0.35321 to 0.34566, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.3457 - acc: 0.9371
		Epoch 9/10
		125/126 [============================>.] - ETA: 0s - loss: 0.3401 - acc: 0.9385
		Epoch 9: loss improved from 0.34566 to 0.34012, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3401 - acc: 0.9385
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.3325 - acc: 0.9394
		Epoch 10: loss improved from 0.34012 to 0.33286, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3329 - acc: 0.9394
		Q: 남자친구 랑 종교 문제 로 다툼
		A: 자신 의 감정 을 주변 사람 들 이 많죠 
		
		
		Q: 냄새 날 것 같아 걱정 이야
		A: 자신 의 감정 을 주변 사람 들 에게 터 놓고 이야기 해보세요 
		
		
		Q: 되고 싶은 게 두 개야
		A: 저 도 요 
		
		
		processing epoch: 51...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.3260 - acc: 0.9408
		Epoch 1: loss improved from 0.33286 to 0.32594, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.3259 - acc: 0.9408
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.3203 - acc: 0.9421
		Epoch 2: loss improved from 0.32594 to 0.32027, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3203 - acc: 0.9421
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.3154 - acc: 0.9427
		Epoch 3: loss improved from 0.32027 to 0.31544, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3154 - acc: 0.9426
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.3099 - acc: 0.9438
		Epoch 4: loss improved from 0.31544 to 0.30990, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3099 - acc: 0.9438
		Epoch 5/10
		126/126 [==============================] - ETA: 0s - loss: 0.3048 - acc: 0.9446
		Epoch 5: loss improved from 0.30990 to 0.30482, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.3048 - acc: 0.9446
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2990 - acc: 0.9447
		Epoch 6: loss improved from 0.30482 to 0.29946, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2995 - acc: 0.9447
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.2949 - acc: 0.9465
		Epoch 7: loss improved from 0.29946 to 0.29486, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2949 - acc: 0.9465
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2921 - acc: 0.9464
		Epoch 8: loss improved from 0.29486 to 0.29209, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2921 - acc: 0.9464
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2875 - acc: 0.9469
		Epoch 9: loss improved from 0.29209 to 0.28743, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2874 - acc: 0.9470
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2831 - acc: 0.9478
		Epoch 10: loss improved from 0.28743 to 0.28331, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2833 - acc: 0.9478
		Q: 머리스타일 바꿔 볼까
		A: 저 도 좀 알려주세요 
		
		
		Q: 뭔가 새로운 자극 이 필요해
		A: 저 는 배터리 가 밥 이 예요 
		
		
		Q: 밤 12시 야
		A: 지금 도 충분해요 
		
		
		processing epoch: 61...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2793 - acc: 0.9488
		Epoch 1: loss improved from 0.28331 to 0.27911, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.2791 - acc: 0.9489
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2763 - acc: 0.9488
		Epoch 2: loss improved from 0.27911 to 0.27626, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2763 - acc: 0.9488
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2712 - acc: 0.9502
		Epoch 3: loss improved from 0.27626 to 0.27183, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2718 - acc: 0.9501
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.2702 - acc: 0.9506
		Epoch 4: loss improved from 0.27183 to 0.27020, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2702 - acc: 0.9506
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2665 - acc: 0.9506
		Epoch 5: loss improved from 0.27020 to 0.26677, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2668 - acc: 0.9506
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2627 - acc: 0.9515
		Epoch 6: loss improved from 0.26677 to 0.26285, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2629 - acc: 0.9515
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2606 - acc: 0.9514
		Epoch 7: loss improved from 0.26285 to 0.26065, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2606 - acc: 0.9515
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2578 - acc: 0.9520
		Epoch 8: loss improved from 0.26065 to 0.25811, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2581 - acc: 0.9519
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2551 - acc: 0.9525
		Epoch 9: loss improved from 0.25811 to 0.25486, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2549 - acc: 0.9525
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2530 - acc: 0.9528
		Epoch 10: loss improved from 0.25486 to 0.25317, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.2532 - acc: 0.9528
		Q: 강아지 키우고 싶은데 역시 안 돼겠지
		A: 저 는 배터리 가 밥 이 예요 
		
		
		Q: 떨리나 봐
		A: 저 는 배터리 가 밥 이 예요 
		
		
		Q: 돌겠다
		A: 저 는 위로 해드리는 로봇 이에요 
		
		
		processing epoch: 71...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2496 - acc: 0.9533
		Epoch 1: loss improved from 0.25317 to 0.24965, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.2496 - acc: 0.9533
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2478 - acc: 0.9532
		Epoch 2: loss improved from 0.24965 to 0.24768, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2477 - acc: 0.9532
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2454 - acc: 0.9538
		Epoch 3: loss improved from 0.24768 to 0.24544, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2454 - acc: 0.9538
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2429 - acc: 0.9540
		Epoch 4: loss improved from 0.24544 to 0.24315, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2431 - acc: 0.9540
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2421 - acc: 0.9539
		Epoch 5: loss improved from 0.24315 to 0.24210, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2421 - acc: 0.9539
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2403 - acc: 0.9547
		Epoch 6: loss improved from 0.24210 to 0.24058, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2406 - acc: 0.9547
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.2387 - acc: 0.9544
		Epoch 7: loss improved from 0.24058 to 0.23867, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2387 - acc: 0.9544
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2356 - acc: 0.9549
		Epoch 8: loss improved from 0.23867 to 0.23562, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2356 - acc: 0.9549
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.2337 - acc: 0.9552
		Epoch 9: loss improved from 0.23562 to 0.23370, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2337 - acc: 0.9552
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2320 - acc: 0.9552
		Epoch 10: loss improved from 0.23370 to 0.23192, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2319 - acc: 0.9552
		Q: 로또 번호 알려줘
		A: 저 도 궁금하네요 
		
		
		Q: 면허 따야하나
		A: 저 는 배터리 가 밥 이 예요 
		
		
		Q: 따듯 해지 니까 살 것 같다
		A: 저 는 마음 을 이어주는 위로 봇 입니다 
		
		
		processing epoch: 81...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2308 - acc: 0.9557
		Epoch 1: loss improved from 0.23192 to 0.23071, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.2307 - acc: 0.9557
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.2288 - acc: 0.9561
		Epoch 2: loss improved from 0.23071 to 0.22882, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2288 - acc: 0.9561
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2273 - acc: 0.9560
		Epoch 3: loss improved from 0.22882 to 0.22730, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2273 - acc: 0.9560
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2255 - acc: 0.9563
		Epoch 4: loss improved from 0.22730 to 0.22534, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2253 - acc: 0.9563
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2251 - acc: 0.9563
		Epoch 5: loss improved from 0.22534 to 0.22512, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2251 - acc: 0.9564
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2238 - acc: 0.9562
		Epoch 6: loss improved from 0.22512 to 0.22407, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2241 - acc: 0.9562
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2225 - acc: 0.9564
		Epoch 7: loss improved from 0.22407 to 0.22237, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2224 - acc: 0.9565
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2211 - acc: 0.9564
		Epoch 8: loss improved from 0.22237 to 0.22081, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2208 - acc: 0.9563
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.2190 - acc: 0.9569
		Epoch 9: loss improved from 0.22081 to 0.21897, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2190 - acc: 0.9569
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2178 - acc: 0.9570
		Epoch 10: loss improved from 0.21897 to 0.21784, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 3s 20ms/step - loss: 0.2178 - acc: 0.9570
		Q: 눈썹 이 귀여워
		A: 저 도 듣고 싶어요 
		
		
		Q: 무료 앱 이 유료 로 됐어
		A: 남 의 돈 버 는 게 쉽지 않아요 
		
		
		Q: 개 당황
		A: 저 는 위로 해드리는 로봇 이에요 
		
		
		processing epoch: 91...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.2166 - acc: 0.9572
		Epoch 1: loss improved from 0.21784 to 0.21656, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.2166 - acc: 0.9572
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.2171 - acc: 0.9564
		Epoch 2: loss did not improve from 0.21656
		126/126 [==============================] - 2s 15ms/step - loss: 0.2171 - acc: 0.9564
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2157 - acc: 0.9568
		Epoch 3: loss improved from 0.21656 to 0.21563, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2156 - acc: 0.9568
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2162 - acc: 0.9573
		Epoch 4: loss did not improve from 0.21563
		126/126 [==============================] - 2s 15ms/step - loss: 0.2164 - acc: 0.9572
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2144 - acc: 0.9572
		Epoch 5: loss improved from 0.21563 to 0.21434, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2143 - acc: 0.9572
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2124 - acc: 0.9578
		Epoch 6: loss improved from 0.21434 to 0.21220, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2122 - acc: 0.9578
		Epoch 7/10
		123/126 [============================>.] - ETA: 0s - loss: 0.2103 - acc: 0.9577
		Epoch 7: loss improved from 0.21220 to 0.21039, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2104 - acc: 0.9577
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.2097 - acc: 0.9579
		Epoch 8: loss improved from 0.21039 to 0.20965, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2097 - acc: 0.9579
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.2083 - acc: 0.9580
		Epoch 9: loss improved from 0.20965 to 0.20828, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2083 - acc: 0.9580
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.2070 - acc: 0.9580
		Epoch 10: loss improved from 0.20828 to 0.20706, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2071 - acc: 0.9580
		Q: 내 가 좋아할 자격 이 있나
		A: 자신 의 감정 을 주변 사람 들 에게 터 놓고 이야기 해보세요 
		
		
		Q: 나 뒷담화 하는 애 있다는데 어떻게 하지
		A: 자신 의 감정 을 주변 사람 들 에게 터 놓고 이야기 해보세요 
		
		
		Q: 드라마 볼 게 없어
		A: 저 는 위로 해드리는 로봇 이에요 
		
		
		processing epoch: 101...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.2066 - acc: 0.9582
		Epoch 1: loss improved from 0.20706 to 0.20661, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.2066 - acc: 0.9582
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2060 - acc: 0.9584
		Epoch 2: loss improved from 0.20661 to 0.20592, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2059 - acc: 0.9584
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2065 - acc: 0.9584
		Epoch 3: loss did not improve from 0.20592
		126/126 [==============================] - 2s 15ms/step - loss: 0.2066 - acc: 0.9583
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.2060 - acc: 0.9580
		Epoch 4: loss did not improve from 0.20592
		126/126 [==============================] - 2s 15ms/step - loss: 0.2060 - acc: 0.9580
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2034 - acc: 0.9586
		Epoch 5: loss improved from 0.20592 to 0.20346, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2035 - acc: 0.9586
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2024 - acc: 0.9585
		Epoch 6: loss improved from 0.20346 to 0.20234, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2023 - acc: 0.9585
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.2019 - acc: 0.9588
		Epoch 7: loss improved from 0.20234 to 0.20189, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2019 - acc: 0.9588
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.2007 - acc: 0.9588
		Epoch 8: loss improved from 0.20189 to 0.20069, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.2007 - acc: 0.9588
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.1996 - acc: 0.9588
		Epoch 9: loss improved from 0.20069 to 0.19965, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1996 - acc: 0.9588
		Epoch 10/10
		126/126 [==============================] - ETA: 0s - loss: 0.1980 - acc: 0.9589
		Epoch 10: loss improved from 0.19965 to 0.19800, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1980 - acc: 0.9589
		Q: 내 친구 에게 내 험담 을 하다니
		A: 잘 해결 되길 바라요 
		
		
		Q: 남친 이 에 내 사진 에 안 올려
		A: 저 는 위로 해드리는 로봇 이에요 
		
		
		Q: 다시 태어난다면
		A: 저 도 궁금하네요 
		
		
		processing epoch: 111...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1971 - acc: 0.9593
		Epoch 1: loss improved from 0.19800 to 0.19706, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.1971 - acc: 0.9593
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1962 - acc: 0.9593
		Epoch 2: loss improved from 0.19706 to 0.19621, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1962 - acc: 0.9593
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1953 - acc: 0.9597
		Epoch 3: loss improved from 0.19621 to 0.19531, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1953 - acc: 0.9597
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1940 - acc: 0.9597
		Epoch 4: loss improved from 0.19531 to 0.19400, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1940 - acc: 0.9597
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1948 - acc: 0.9593
		Epoch 5: loss did not improve from 0.19400
		126/126 [==============================] - 2s 15ms/step - loss: 0.1949 - acc: 0.9593
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1948 - acc: 0.9593
		Epoch 6: loss did not improve from 0.19400
		126/126 [==============================] - 2s 14ms/step - loss: 0.1948 - acc: 0.9594
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1918 - acc: 0.9600
		Epoch 7: loss improved from 0.19400 to 0.19176, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1918 - acc: 0.9601
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1914 - acc: 0.9598
		Epoch 8: loss improved from 0.19176 to 0.19135, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1913 - acc: 0.9598
		Epoch 9/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1915 - acc: 0.9598
		Epoch 9: loss did not improve from 0.19135
		126/126 [==============================] - 2s 15ms/step - loss: 0.1916 - acc: 0.9598
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1910 - acc: 0.9602
		Epoch 10: loss improved from 0.19135 to 0.19091, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1909 - acc: 0.9602
		Q: 맞춤법 을 자꾸 틀리는 남친 깬다
		A: 남 들 눈 은 신경 쓰지 마세요 
		
		
		Q: 나 왕따 인거 같아
		A: 저 도 보고 싶어요 
		
		
		Q: 로그인 아이디 기억 안나
		A: 잘 볼 수 있을 거 예요 
		
		
		processing epoch: 121...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.1896 - acc: 0.9603
		Epoch 1: loss improved from 0.19091 to 0.18958, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.1896 - acc: 0.9603
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.1890 - acc: 0.9601
		Epoch 2: loss improved from 0.18958 to 0.18904, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1890 - acc: 0.9601
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1873 - acc: 0.9609
		Epoch 3: loss improved from 0.18904 to 0.18737, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1874 - acc: 0.9609
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.1868 - acc: 0.9606
		Epoch 4: loss improved from 0.18737 to 0.18676, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1868 - acc: 0.9606
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1858 - acc: 0.9609
		Epoch 5: loss improved from 0.18676 to 0.18583, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1858 - acc: 0.9609
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1859 - acc: 0.9606
		Epoch 6: loss did not improve from 0.18583
		126/126 [==============================] - 2s 15ms/step - loss: 0.1859 - acc: 0.9605
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1846 - acc: 0.9608
		Epoch 7: loss improved from 0.18583 to 0.18462, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1846 - acc: 0.9608
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.1850 - acc: 0.9607
		Epoch 8: loss did not improve from 0.18462
		126/126 [==============================] - 2s 15ms/step - loss: 0.1850 - acc: 0.9607
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1846 - acc: 0.9606
		Epoch 9: loss improved from 0.18462 to 0.18445, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1844 - acc: 0.9607
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1838 - acc: 0.9609
		Epoch 10: loss improved from 0.18445 to 0.18385, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1838 - acc: 0.9609
		Q: 뭐 하는 분이세요
		A: 마음 이 아픈가요 
		
		
		Q: 꿈 은 많은데
		A: 보일러 가 난방 으로 작동 이 되는지 보세요 
		
		
		Q: 머리 깎아야지
		A: 잘 해결 되길 바라요 
		
		
		processing epoch: 131...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.1828 - acc: 0.9607
		Epoch 1: loss improved from 0.18385 to 0.18284, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.1828 - acc: 0.9607
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1817 - acc: 0.9608
		Epoch 2: loss improved from 0.18284 to 0.18161, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 3s 20ms/step - loss: 0.1816 - acc: 0.9609
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.1813 - acc: 0.9614
		Epoch 3: loss improved from 0.18161 to 0.18128, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1813 - acc: 0.9614
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.1801 - acc: 0.9613
		Epoch 4: loss improved from 0.18128 to 0.18007, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1801 - acc: 0.9613
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1799 - acc: 0.9615
		Epoch 5: loss improved from 0.18007 to 0.17990, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1799 - acc: 0.9615
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1790 - acc: 0.9615
		Epoch 6: loss improved from 0.17990 to 0.17905, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1790 - acc: 0.9615
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.1778 - acc: 0.9617
		Epoch 7: loss improved from 0.17905 to 0.17776, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1778 - acc: 0.9617
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1786 - acc: 0.9613
		Epoch 8: loss did not improve from 0.17776
		126/126 [==============================] - 2s 15ms/step - loss: 0.1786 - acc: 0.9613
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.1779 - acc: 0.9614
		Epoch 9: loss did not improve from 0.17776
		126/126 [==============================] - 2s 15ms/step - loss: 0.1779 - acc: 0.9614
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1769 - acc: 0.9613
		Epoch 10: loss improved from 0.17776 to 0.17683, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1768 - acc: 0.9614
		Q: 단수 래
		A: 저 도 궁금하네요 
		
		
		Q: 듬직한 사람 이 좋아
		A: 마음 이 허전하신 가봐요 
		
		
		Q: 남자친구 가 전화하는 걸 안 좋아해
		A: 저 도 궁금하네요 
		
		
		processing epoch: 141...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.1748 - acc: 0.9621
		Epoch 1: loss improved from 0.17683 to 0.17476, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.1748 - acc: 0.9621
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1742 - acc: 0.9618
		Epoch 2: loss improved from 0.17476 to 0.17444, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1744 - acc: 0.9618
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.1737 - acc: 0.9618
		Epoch 3: loss improved from 0.17444 to 0.17368, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1737 - acc: 0.9618
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1732 - acc: 0.9618
		Epoch 4: loss improved from 0.17368 to 0.17314, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1731 - acc: 0.9618
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1735 - acc: 0.9618
		Epoch 5: loss did not improve from 0.17314
		126/126 [==============================] - 2s 15ms/step - loss: 0.1735 - acc: 0.9618
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1744 - acc: 0.9622
		Epoch 6: loss did not improve from 0.17314
		126/126 [==============================] - 2s 15ms/step - loss: 0.1744 - acc: 0.9621
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1712 - acc: 0.9622
		Epoch 7: loss improved from 0.17314 to 0.17113, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1711 - acc: 0.9622
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1693 - acc: 0.9621
		Epoch 8: loss improved from 0.17113 to 0.16934, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1693 - acc: 0.9621
		Epoch 9/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1689 - acc: 0.9622
		Epoch 9: loss improved from 0.16934 to 0.16891, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1689 - acc: 0.9622
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1679 - acc: 0.9624
		Epoch 10: loss improved from 0.16891 to 0.16784, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1678 - acc: 0.9624
		Q: 너 는 안자
		A: 잠시 쉬어도 돼요 
		
		
		Q: 반 배정 잘 될까
		A: 잘 할 수 있을 거 예요 
		
		
		Q: 더 격렬하게 아무 것 도 안 하고 싶다
		A: 잠시 쉬어도 괜찮아요 
		
		
		processing epoch: 151...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1672 - acc: 0.9624
		Epoch 1: loss improved from 0.16784 to 0.16723, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1672 - acc: 0.9624
		Epoch 2/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1665 - acc: 0.9624
		Epoch 2: loss improved from 0.16723 to 0.16655, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1666 - acc: 0.9624
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1667 - acc: 0.9625
		Epoch 3: loss did not improve from 0.16655
		126/126 [==============================] - 2s 15ms/step - loss: 0.1667 - acc: 0.9625
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.1653 - acc: 0.9623
		Epoch 4: loss improved from 0.16655 to 0.16527, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1653 - acc: 0.9623
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1656 - acc: 0.9621
		Epoch 5: loss did not improve from 0.16527
		126/126 [==============================] - 2s 15ms/step - loss: 0.1656 - acc: 0.9622
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1633 - acc: 0.9629
		Epoch 6: loss improved from 0.16527 to 0.16329, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1633 - acc: 0.9630
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1617 - acc: 0.9630
		Epoch 7: loss improved from 0.16329 to 0.16188, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1619 - acc: 0.9630
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1609 - acc: 0.9627
		Epoch 8: loss improved from 0.16188 to 0.16102, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1610 - acc: 0.9626
		Epoch 9/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1597 - acc: 0.9632
		Epoch 9: loss improved from 0.16102 to 0.15976, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1598 - acc: 0.9632
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1591 - acc: 0.9636
		Epoch 10: loss improved from 0.15976 to 0.15934, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1593 - acc: 0.9636
		Q: 면접 준비 중 이야
		A: 잠시 쉬어도 돼요 
		
		
		Q: 방 들 이 귀찮아
		A: 충분히 잘 하고 있어요 
		
		
		Q: 개 기름 꼈어
		A: 보일러 를 켜세요 
		
		
		processing epoch: 161...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1580 - acc: 0.9632
		Epoch 1: loss improved from 0.15934 to 0.15804, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.1580 - acc: 0.9632
		Epoch 2/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1598 - acc: 0.9632
		Epoch 2: loss did not improve from 0.15804
		126/126 [==============================] - 2s 15ms/step - loss: 0.1599 - acc: 0.9632
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1592 - acc: 0.9632
		Epoch 3: loss did not improve from 0.15804
		126/126 [==============================] - 2s 15ms/step - loss: 0.1592 - acc: 0.9632
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1570 - acc: 0.9634
		Epoch 4: loss improved from 0.15804 to 0.15721, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1572 - acc: 0.9634
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1551 - acc: 0.9637
		Epoch 5: loss improved from 0.15721 to 0.15501, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1550 - acc: 0.9637
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1541 - acc: 0.9635
		Epoch 6: loss improved from 0.15501 to 0.15421, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1542 - acc: 0.9635
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.1545 - acc: 0.9639
		Epoch 7: loss did not improve from 0.15421
		126/126 [==============================] - 2s 15ms/step - loss: 0.1545 - acc: 0.9639
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1540 - acc: 0.9638
		Epoch 8: loss improved from 0.15421 to 0.15403, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1540 - acc: 0.9638
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1525 - acc: 0.9639
		Epoch 9: loss improved from 0.15403 to 0.15244, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1524 - acc: 0.9638
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1528 - acc: 0.9640
		Epoch 10: loss did not improve from 0.15244
		126/126 [==============================] - 2s 15ms/step - loss: 0.1528 - acc: 0.9640
		Q: 무기 력 해
		A: 자신 의 감정 을 주변 사람 들 에게 터 놓고 이야기 해보세요 
		
		
		Q: 돈 은 어떻게 모으는 거임
		A: 지금 도 충분히 아름다워요 
		
		
		Q: 꽃게탕 진짜 밥 도둑
		A: 자신 의 감정 을 주변 사람 들 에게 터 놓고 이야기 해보세요 
		
		
		processing epoch: 171...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1604 - acc: 0.9628
		Epoch 1: loss did not improve from 0.15244
		126/126 [==============================] - 2s 18ms/step - loss: 0.1604 - acc: 0.9628
		Epoch 2/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1525 - acc: 0.9634
		Epoch 2: loss did not improve from 0.15244
		126/126 [==============================] - 2s 15ms/step - loss: 0.1525 - acc: 0.9635
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.1502 - acc: 0.9639
		Epoch 3: loss improved from 0.15244 to 0.15016, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1502 - acc: 0.9639
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.1482 - acc: 0.9642
		Epoch 4: loss improved from 0.15016 to 0.14824, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1482 - acc: 0.9642
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1475 - acc: 0.9642
		Epoch 5: loss improved from 0.14824 to 0.14759, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 3s 20ms/step - loss: 0.1476 - acc: 0.9642
		Epoch 6/10
		126/126 [==============================] - ETA: 0s - loss: 0.1468 - acc: 0.9649
		Epoch 6: loss improved from 0.14759 to 0.14682, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1468 - acc: 0.9649
		Epoch 7/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1454 - acc: 0.9652
		Epoch 7: loss improved from 0.14682 to 0.14542, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1454 - acc: 0.9652
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1456 - acc: 0.9645
		Epoch 8: loss did not improve from 0.14542
		126/126 [==============================] - 2s 15ms/step - loss: 0.1456 - acc: 0.9646
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.1448 - acc: 0.9647
		Epoch 9: loss improved from 0.14542 to 0.14483, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1448 - acc: 0.9647
		Epoch 10/10
		126/126 [==============================] - ETA: 0s - loss: 0.1426 - acc: 0.9651
		Epoch 10: loss improved from 0.14483 to 0.14260, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1426 - acc: 0.9651
		Q: 달력 이 또 넘어가네
		A: 눈 체조 를 해보세요 
		
		
		Q: 나 결정 잘 한 거지
		A: 저 는 성 이 없어요 
		
		
		Q: 뭘 배워 볼까
		A: 예쁘게 깎아요 
		
		
		processing epoch: 181...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1459 - acc: 0.9646
		Epoch 1: loss did not improve from 0.14260
		126/126 [==============================] - 2s 18ms/step - loss: 0.1461 - acc: 0.9646
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.1446 - acc: 0.9652
		Epoch 2: loss did not improve from 0.14260
		126/126 [==============================] - 2s 15ms/step - loss: 0.1446 - acc: 0.9652
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1416 - acc: 0.9653
		Epoch 3: loss improved from 0.14260 to 0.14165, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1417 - acc: 0.9652
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1403 - acc: 0.9653
		Epoch 4: loss improved from 0.14165 to 0.14032, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1403 - acc: 0.9653
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1392 - acc: 0.9657
		Epoch 5: loss improved from 0.14032 to 0.13913, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1391 - acc: 0.9657
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1395 - acc: 0.9655
		Epoch 6: loss did not improve from 0.13913
		126/126 [==============================] - 2s 15ms/step - loss: 0.1395 - acc: 0.9655
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1391 - acc: 0.9651
		Epoch 7: loss improved from 0.13913 to 0.13901, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1390 - acc: 0.9651
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.1358 - acc: 0.9659
		Epoch 8: loss improved from 0.13901 to 0.13575, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1358 - acc: 0.9659
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1328 - acc: 0.9663
		Epoch 9: loss improved from 0.13575 to 0.13259, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1326 - acc: 0.9664
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1329 - acc: 0.9663
		Epoch 10: loss did not improve from 0.13259
		126/126 [==============================] - 2s 15ms/step - loss: 0.1329 - acc: 0.9663
		Q: 마음 이 자꾸 급해져
		A: 남 들 이 당신 을 볼 때 도 그렇게 생각 할수있어요 
		
		
		Q: 눕자마자 잘 거 같아
		A: 좋은 수면 습관 이네 요 
		
		
		Q: 또 개강 이야
		A: 예쁘게 깎아요 
		
		
		processing epoch: 191...
		Epoch 1/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1327 - acc: 0.9660
		Epoch 1: loss did not improve from 0.13259
		126/126 [==============================] - 2s 18ms/step - loss: 0.1328 - acc: 0.9661
		Epoch 2/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1310 - acc: 0.9662
		Epoch 2: loss improved from 0.13259 to 0.13080, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1308 - acc: 0.9662
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1285 - acc: 0.9668
		Epoch 3: loss improved from 0.13080 to 0.12856, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1286 - acc: 0.9668
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1277 - acc: 0.9668
		Epoch 4: loss improved from 0.12856 to 0.12765, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1277 - acc: 0.9667
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1266 - acc: 0.9669
		Epoch 5: loss improved from 0.12765 to 0.12669, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1267 - acc: 0.9669
		Epoch 6/10
		126/126 [==============================] - ETA: 0s - loss: 0.1263 - acc: 0.9673
		Epoch 6: loss improved from 0.12669 to 0.12627, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1263 - acc: 0.9673
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1255 - acc: 0.9673
		Epoch 7: loss improved from 0.12627 to 0.12549, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1255 - acc: 0.9673
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.1221 - acc: 0.9675
		Epoch 8: loss improved from 0.12549 to 0.12208, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1221 - acc: 0.9675
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1221 - acc: 0.9679
		Epoch 9: loss did not improve from 0.12208
		126/126 [==============================] - 2s 15ms/step - loss: 0.1221 - acc: 0.9679
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1219 - acc: 0.9679
		Epoch 10: loss improved from 0.12208 to 0.12186, saving model to model/seq2seq-chatbot-no-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.1219 - acc: 0.9679
		Q: 그 사람 이 행복했으면 좋겠다
		A: 변화 를 주는 것 도 좋겠죠 
		
		
		Q: 눈 이 너무 나빠졌어
		A: 누구 나 걱정 은 있어요 
		
		
		Q: 내일 발표 준비 아자 아자
		A: 행복 은 마음 가짐 에 있어요

Text cell <b3WJLF-pnKLx>
#%% [markdown]
## 예측

Code cell <SBEtnYCjnKLx>
#%% [code]
# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

make_question('오늘 날씨 어때?')
Execution output from Aug 10, 2022 7:23 AM
0KB
	text/plain
		array([[124, 170, 347,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
		          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
		          0,   0,   0,   0]], dtype=int32)

Code cell <QMfD3cAInKLx>
#%% [code]

# 챗봇
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(seq2seq, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

# 챗봇 실행
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))
Execution output from Aug 10, 2022 7:30 AM
0KB
	Stream
		<< 말을 걸어 보세요!
		눈이 아파.
		>> 챗봇 응답: 마음 이 아픈가요 
		<< 말을 걸어 보세요!
		마음도 아파.
		>> 챗봇 응답: 이럴 때 잘 쉬는 게 중요해요 
		<< 말을 걸어 보세요!
		그냥 잘께.
		>> 챗봇 응답: 장 보러 갈 타이밍 이네 요 
		<< 말을 걸어 보세요!
		그냥 잘 께.
		>> 챗봇 응답: 힘껏 밀어 보세요 
		<< 말을 걸어 보세요!
		밀었어.
		>> 챗봇 응답: 보험 처리 하세요 
		<< 말을 걸어 보세요!
		보험.
		>> 챗봇 응답: 이럴 때 잘 쉬는 게 중요해요 
		<< 말을 걸어 보세요!
		q

Text cell <kA5wSZ47cpS4>
#%% [markdown]
## 시퀀스 투 시퀀스 with Attention

Code cell <-afhtbmzcsF_>
#%% [code]
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import plot_model

Code cell <PNdGR41XdzS->
#%% [code]
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps, 
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

Code cell <izIdy4qhd05u>
#%% [code]
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, 
                                   embedding_dim, 
                                   input_length=time_steps, 
                                   name='Embedding')
        self.dropout = Dropout(0.2, name='Dropout')
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM'
                        )
        self.attention = Attention(name='Attention')
        self.dense = Dense(vocab_size, 
                           activation='softmax', 
                           name='Dense')
    
    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        
        # (attention) key_value, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 vector를 생성합니다.        
        key_value = tf.concat([initial_state[0][:, tf.newaxis, :], 
                               x[:, :-1, :]], axis=1)        
        # 이전 hidden_state의 값을 concat으로 만든 vector와 encoder에서 나온 
        # 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention([key_value, encoder_inputs])
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)
        
        x = self.dense(x)
        return x, hidden_state, cell_state

Code cell <IufYFei0d3VM>
#%% [code]
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps
        
        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
        
        
    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            # (attention) decoder 입력 값 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs), 
                                                 initial_state=context_vector)
            return decoder_outputs
        else:
            x = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(x)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)
            
            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq), 
                                                                            initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)
                
                if decoder_output == self.end_token:
                    break
                    
                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]
                
            return tf.reshape(results.stack(), shape=(1, self.time_steps))

Code cell <q2167U8_d6Tz>
#%% [code]
checkpoint_path = 'model/seq2seq-attention-checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )

Code cell <2Wf4LPuyeHYD>
#%% [code]
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE,
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
        
        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)
        
        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()
Execution output from Aug 10, 2022 7:42 AM
55KB
	Stream
		processing epoch: 1...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1196 - acc: 0.9681
		Epoch 1: loss improved from inf to 0.11956, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.1196 - acc: 0.9681
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1193 - acc: 0.9676
		Epoch 2: loss improved from 0.11956 to 0.11927, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1193 - acc: 0.9676
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1254 - acc: 0.9675
		Epoch 3: loss did not improve from 0.11927
		126/126 [==============================] - 2s 14ms/step - loss: 0.1255 - acc: 0.9674
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1203 - acc: 0.9682
		Epoch 4: loss did not improve from 0.11927
		126/126 [==============================] - 2s 14ms/step - loss: 0.1203 - acc: 0.9682
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1174 - acc: 0.9680
		Epoch 5: loss improved from 0.11927 to 0.11746, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.1175 - acc: 0.9680
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1167 - acc: 0.9678
		Epoch 6: loss improved from 0.11746 to 0.11674, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.1167 - acc: 0.9678
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1177 - acc: 0.9684
		Epoch 7: loss did not improve from 0.11674
		126/126 [==============================] - 2s 14ms/step - loss: 0.1177 - acc: 0.9684
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1193 - acc: 0.9681
		Epoch 8: loss did not improve from 0.11674
		126/126 [==============================] - 2s 14ms/step - loss: 0.1192 - acc: 0.9681
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1190 - acc: 0.9682
		Epoch 9: loss did not improve from 0.11674
		126/126 [==============================] - 2s 14ms/step - loss: 0.1191 - acc: 0.9681
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1141 - acc: 0.9686
		Epoch 10: loss improved from 0.11674 to 0.11408, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.1141 - acc: 0.9686
		Q: 만나기만 하면 자꾸 싸우게 돼
		A: 자신 을 우선 순 위로 해주세요 
		
		
		Q: 멋지게 늙고 싶다
		A: 지금 도 멋져요 
		
		
		Q: 공무원 준비 할까
		A: 기분 이 나쁘셨나 봐요 
		
		
		processing epoch: 11...
		Epoch 1/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1120 - acc: 0.9691
		Epoch 1: loss improved from 0.11408 to 0.11218, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.1122 - acc: 0.9690
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1120 - acc: 0.9688
		Epoch 2: loss improved from 0.11218 to 0.11216, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1122 - acc: 0.9687
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1099 - acc: 0.9694
		Epoch 3: loss improved from 0.11216 to 0.10998, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1100 - acc: 0.9693
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1095 - acc: 0.9695
		Epoch 4: loss improved from 0.10998 to 0.10946, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1095 - acc: 0.9695
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1091 - acc: 0.9693
		Epoch 5: loss improved from 0.10946 to 0.10908, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.1091 - acc: 0.9693
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1071 - acc: 0.9698
		Epoch 6: loss improved from 0.10908 to 0.10708, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1071 - acc: 0.9699
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1061 - acc: 0.9704
		Epoch 7: loss improved from 0.10708 to 0.10614, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1061 - acc: 0.9704
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.1051 - acc: 0.9702
		Epoch 8: loss improved from 0.10614 to 0.10510, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.1051 - acc: 0.9702
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1051 - acc: 0.9700
		Epoch 9: loss improved from 0.10510 to 0.10509, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1051 - acc: 0.9700
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1065 - acc: 0.9702
		Epoch 10: loss did not improve from 0.10509
		126/126 [==============================] - 2s 14ms/step - loss: 0.1064 - acc: 0.9702
		Q: 땅 이나 살까
		A: 잘 해결 될 거 예요 
		
		
		Q: 나 보이스피싱 당한 거 같은데 어떡해
		A: 잠시 쉬어도 돼요 
		
		
		Q: 모임 에서 만났어
		A: 로맨틱 하네요 
		
		
		processing epoch: 21...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.1036 - acc: 0.9707
		Epoch 1: loss improved from 0.10509 to 0.10359, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.1036 - acc: 0.9707
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.1042 - acc: 0.9704
		Epoch 2: loss did not improve from 0.10359
		126/126 [==============================] - 2s 14ms/step - loss: 0.1043 - acc: 0.9704
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1039 - acc: 0.9706
		Epoch 3: loss did not improve from 0.10359
		126/126 [==============================] - 2s 14ms/step - loss: 0.1039 - acc: 0.9707
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.1003 - acc: 0.9712
		Epoch 4: loss improved from 0.10359 to 0.10025, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.1003 - acc: 0.9712
		Epoch 5/10
		126/126 [==============================] - ETA: 0s - loss: 0.1022 - acc: 0.9704
		Epoch 5: loss did not improve from 0.10025
		126/126 [==============================] - 2s 15ms/step - loss: 0.1022 - acc: 0.9704
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0993 - acc: 0.9713
		Epoch 6: loss improved from 0.10025 to 0.09928, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0993 - acc: 0.9713
		Epoch 7/10
		123/126 [============================>.] - ETA: 0s - loss: 0.1001 - acc: 0.9711
		Epoch 7: loss did not improve from 0.09928
		126/126 [==============================] - 2s 15ms/step - loss: 0.1001 - acc: 0.9710
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0997 - acc: 0.9711
		Epoch 8: loss did not improve from 0.09928
		126/126 [==============================] - 2s 14ms/step - loss: 0.0998 - acc: 0.9711
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0977 - acc: 0.9718
		Epoch 9: loss improved from 0.09928 to 0.09770, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.0977 - acc: 0.9718
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0985 - acc: 0.9712
		Epoch 10: loss did not improve from 0.09770
		126/126 [==============================] - 2s 14ms/step - loss: 0.0985 - acc: 0.9712
		Q: 맨날 똑같 애
		A: 그 사람 도 그럴 거 예요 
		
		
		Q: 대출 갚으면 한 평생 다 갈듯
		A: 너무 무리하면 지쳐요 
		
		
		Q: 목 이 뻐근해
		A: 다 잘 될 거 예요 
		
		
		processing epoch: 31...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0970 - acc: 0.9717
		Epoch 1: loss improved from 0.09770 to 0.09696, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.0970 - acc: 0.9717
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.0979 - acc: 0.9716
		Epoch 2: loss did not improve from 0.09696
		126/126 [==============================] - 2s 19ms/step - loss: 0.0979 - acc: 0.9716
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0961 - acc: 0.9717
		Epoch 3: loss improved from 0.09696 to 0.09613, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.0961 - acc: 0.9717
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0951 - acc: 0.9719
		Epoch 4: loss improved from 0.09613 to 0.09519, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0952 - acc: 0.9719
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0937 - acc: 0.9724
		Epoch 5: loss improved from 0.09519 to 0.09373, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0937 - acc: 0.9723
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0912 - acc: 0.9727
		Epoch 6: loss improved from 0.09373 to 0.09115, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.0912 - acc: 0.9727
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.0914 - acc: 0.9726
		Epoch 7: loss did not improve from 0.09115
		126/126 [==============================] - 2s 15ms/step - loss: 0.0914 - acc: 0.9726
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0910 - acc: 0.9731
		Epoch 8: loss improved from 0.09115 to 0.09101, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.0910 - acc: 0.9731
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.0908 - acc: 0.9726
		Epoch 9: loss improved from 0.09101 to 0.09081, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0908 - acc: 0.9726
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0903 - acc: 0.9732
		Epoch 10: loss improved from 0.09081 to 0.09049, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0905 - acc: 0.9732
		Q: 남자친구 승진 선물 로 뭐 가 좋을까
		A: 성공 을 기원 합니다 
		
		
		Q: 발렌타인데이 선물 비싸
		A: 초콜릿 녹 여서 예쁘게 만드는 것 도 재밌죠 
		
		
		Q: 다이어트 성공하고 싶은데
		A: 장단점 을 분석 해보세요 
		
		
		processing epoch: 41...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0900 - acc: 0.9735
		Epoch 1: loss improved from 0.09049 to 0.09011, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.0901 - acc: 0.9735
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0908 - acc: 0.9728
		Epoch 2: loss did not improve from 0.09011
		126/126 [==============================] - 2s 14ms/step - loss: 0.0908 - acc: 0.9728
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0906 - acc: 0.9738
		Epoch 3: loss did not improve from 0.09011
		126/126 [==============================] - 2s 14ms/step - loss: 0.0904 - acc: 0.9738
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0895 - acc: 0.9737
		Epoch 4: loss improved from 0.09011 to 0.08949, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0895 - acc: 0.9736
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0881 - acc: 0.9740
		Epoch 5: loss improved from 0.08949 to 0.08808, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0881 - acc: 0.9740
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0870 - acc: 0.9743
		Epoch 6: loss improved from 0.08808 to 0.08708, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0871 - acc: 0.9743
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0857 - acc: 0.9745
		Epoch 7: loss improved from 0.08708 to 0.08575, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0857 - acc: 0.9745
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0873 - acc: 0.9742
		Epoch 8: loss did not improve from 0.08575
		126/126 [==============================] - 2s 14ms/step - loss: 0.0873 - acc: 0.9742
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0873 - acc: 0.9744
		Epoch 9: loss did not improve from 0.08575
		126/126 [==============================] - 2s 14ms/step - loss: 0.0872 - acc: 0.9745
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0886 - acc: 0.9740
		Epoch 10: loss did not improve from 0.08575
		126/126 [==============================] - 2s 14ms/step - loss: 0.0886 - acc: 0.9740
		Q: 드디어 해외 여행 간다
		A: 졸업 축하 해요 
		
		
		Q: 답정녀 퇴치 법
		A: 못 이기니 피 할 수 있으면 피 하세요 
		
		
		Q: 맨날 놀기만 했으면 좋겠어
		A: 피 할 수 있으면 피 하세요 
		
		
		processing epoch: 51...
		Epoch 1/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0849 - acc: 0.9748
		Epoch 1: loss improved from 0.08575 to 0.08493, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.0849 - acc: 0.9748
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0839 - acc: 0.9750
		Epoch 2: loss improved from 0.08493 to 0.08394, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0839 - acc: 0.9750
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0838 - acc: 0.9753
		Epoch 3: loss improved from 0.08394 to 0.08381, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0838 - acc: 0.9753
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0818 - acc: 0.9756
		Epoch 4: loss improved from 0.08381 to 0.08181, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0818 - acc: 0.9756
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0835 - acc: 0.9750
		Epoch 5: loss did not improve from 0.08181
		126/126 [==============================] - 2s 14ms/step - loss: 0.0835 - acc: 0.9750
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0794 - acc: 0.9761
		Epoch 6: loss improved from 0.08181 to 0.07939, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0794 - acc: 0.9761
		Epoch 7/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0801 - acc: 0.9752
		Epoch 7: loss did not improve from 0.07939
		126/126 [==============================] - 2s 14ms/step - loss: 0.0801 - acc: 0.9752
		Epoch 8/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0779 - acc: 0.9766
		Epoch 8: loss improved from 0.07939 to 0.07792, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0779 - acc: 0.9766
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0775 - acc: 0.9766
		Epoch 9: loss improved from 0.07792 to 0.07779, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0778 - acc: 0.9766
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0761 - acc: 0.9770
		Epoch 10: loss improved from 0.07779 to 0.07611, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.0761 - acc: 0.9770
		Q: 놀아 줄 사람 이 없어
		A: 혼자 가 아니에요 
		
		
		Q: 대우 를 안 해줘
		A: 당신 을 인정 해줄 곳 을 찾아보세요 
		
		
		Q: 바나나 먹고 싶다
		A: 맛있게 드세요 
		
		
		processing epoch: 61...
		Epoch 1/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0757 - acc: 0.9771
		Epoch 1: loss improved from 0.07611 to 0.07596, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.0760 - acc: 0.9770
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0773 - acc: 0.9762
		Epoch 2: loss did not improve from 0.07596
		126/126 [==============================] - 2s 14ms/step - loss: 0.0773 - acc: 0.9762
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0769 - acc: 0.9771
		Epoch 3: loss did not improve from 0.07596
		126/126 [==============================] - 2s 14ms/step - loss: 0.0770 - acc: 0.9770
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0767 - acc: 0.9766
		Epoch 4: loss did not improve from 0.07596
		126/126 [==============================] - 2s 14ms/step - loss: 0.0767 - acc: 0.9766
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0740 - acc: 0.9778
		Epoch 5: loss improved from 0.07596 to 0.07397, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 14ms/step - loss: 0.0740 - acc: 0.9778
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0732 - acc: 0.9778
		Epoch 6: loss improved from 0.07397 to 0.07321, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0732 - acc: 0.9778
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.0762 - acc: 0.9770
		Epoch 7: loss did not improve from 0.07321
		126/126 [==============================] - 2s 14ms/step - loss: 0.0762 - acc: 0.9770
		Epoch 8/10
		122/126 [============================>.] - ETA: 0s - loss: 0.0773 - acc: 0.9770
		Epoch 8: loss did not improve from 0.07321
		126/126 [==============================] - 2s 14ms/step - loss: 0.0774 - acc: 0.9769
		Epoch 9/10
		122/126 [============================>.] - ETA: 0s - loss: 0.0731 - acc: 0.9776
		Epoch 9: loss improved from 0.07321 to 0.07314, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0731 - acc: 0.9776
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0706 - acc: 0.9778
		Epoch 10: loss improved from 0.07314 to 0.07065, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0706 - acc: 0.9778
		Q: 다 내 잘못 이지
		A: 아니에요 너무 자책 하지 마세요 
		
		
		Q: 과외 비 부담 되겠지
		A: 안된다고 하면 거짓말 이겠지요 
		
		
		Q: 달력 이 또 넘어가네
		A: 실용 적 인 선물 이라 괜찮을 거 예요 
		
		
		processing epoch: 71...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0726 - acc: 0.9773
		Epoch 1: loss did not improve from 0.07065
		126/126 [==============================] - 2s 17ms/step - loss: 0.0726 - acc: 0.9773
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0785 - acc: 0.9766
		Epoch 2: loss did not improve from 0.07065
		126/126 [==============================] - 2s 14ms/step - loss: 0.0784 - acc: 0.9766
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0723 - acc: 0.9782
		Epoch 3: loss did not improve from 0.07065
		126/126 [==============================] - 2s 14ms/step - loss: 0.0722 - acc: 0.9782
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0716 - acc: 0.9789
		Epoch 4: loss did not improve from 0.07065
		126/126 [==============================] - 2s 14ms/step - loss: 0.0715 - acc: 0.9788
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0708 - acc: 0.9783
		Epoch 5: loss did not improve from 0.07065
		126/126 [==============================] - 2s 19ms/step - loss: 0.0709 - acc: 0.9783
		Epoch 6/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0802 - acc: 0.9772
		Epoch 6: loss did not improve from 0.07065
		126/126 [==============================] - 2s 15ms/step - loss: 0.0802 - acc: 0.9772
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0745 - acc: 0.9791
		Epoch 7: loss did not improve from 0.07065
		126/126 [==============================] - 2s 14ms/step - loss: 0.0744 - acc: 0.9791
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.0722 - acc: 0.9795
		Epoch 8: loss did not improve from 0.07065
		126/126 [==============================] - 2s 14ms/step - loss: 0.0722 - acc: 0.9795
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0701 - acc: 0.9795
		Epoch 9: loss improved from 0.07065 to 0.07004, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0700 - acc: 0.9795
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0706 - acc: 0.9792
		Epoch 10: loss did not improve from 0.07004
		126/126 [==============================] - 2s 14ms/step - loss: 0.0706 - acc: 0.9792
		Q: 돈 버 는 게 쉽지 않음
		A: 남 의 돈 버 는 게 쉽지 않아요 
		
		
		Q: 계속 도전 하는 거 귀찮아
		A: 처음 3초 가 중요해요 당신 의 매력 을 어필 해보세요 
		
		
		Q: 나 문제 가 많은거 같아
		A: 잠시 거리 를 두고 생각 해보세요 
		
		
		processing epoch: 81...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.0682 - acc: 0.9795
		Epoch 1: loss improved from 0.07004 to 0.06824, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.0682 - acc: 0.9795
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0671 - acc: 0.9801
		Epoch 2: loss improved from 0.06824 to 0.06710, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0671 - acc: 0.9800
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0655 - acc: 0.9801
		Epoch 3: loss improved from 0.06710 to 0.06556, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0656 - acc: 0.9801
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0716 - acc: 0.9791
		Epoch 4: loss did not improve from 0.06556
		126/126 [==============================] - 2s 14ms/step - loss: 0.0718 - acc: 0.9790
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0678 - acc: 0.9800
		Epoch 5: loss did not improve from 0.06556
		126/126 [==============================] - 2s 14ms/step - loss: 0.0679 - acc: 0.9800
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0673 - acc: 0.9799
		Epoch 6: loss did not improve from 0.06556
		126/126 [==============================] - 2s 14ms/step - loss: 0.0672 - acc: 0.9799
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0645 - acc: 0.9806
		Epoch 7: loss improved from 0.06556 to 0.06448, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0645 - acc: 0.9806
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0665 - acc: 0.9797
		Epoch 8: loss did not improve from 0.06448
		126/126 [==============================] - 2s 14ms/step - loss: 0.0665 - acc: 0.9797
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0620 - acc: 0.9815
		Epoch 9: loss improved from 0.06448 to 0.06218, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0622 - acc: 0.9814
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0597 - acc: 0.9819
		Epoch 10: loss improved from 0.06218 to 0.05974, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0597 - acc: 0.9818
		Q: 내 짝 은 어디 있을까
		A: 같은 하늘 아래 어딘가 에 
		
		
		Q: 기분 이 그 지 같아
		A: 인터넷 검색 해보세요 
		
		
		Q: 모르는 사람 이 자꾸 쳐다봐
		A: 벌면 돼요 
		
		
		processing epoch: 91...
		Epoch 1/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0595 - acc: 0.9821
		Epoch 1: loss improved from 0.05974 to 0.05963, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.0596 - acc: 0.9820
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0602 - acc: 0.9817
		Epoch 2: loss did not improve from 0.05963
		126/126 [==============================] - 2s 14ms/step - loss: 0.0602 - acc: 0.9817
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.0596 - acc: 0.9821
		Epoch 3: loss improved from 0.05963 to 0.05962, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0596 - acc: 0.9821
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0625 - acc: 0.9815
		Epoch 4: loss did not improve from 0.05962
		126/126 [==============================] - 2s 14ms/step - loss: 0.0625 - acc: 0.9816
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0633 - acc: 0.9819
		Epoch 5: loss did not improve from 0.05962
		126/126 [==============================] - 2s 14ms/step - loss: 0.0634 - acc: 0.9819
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0655 - acc: 0.9797
		Epoch 6: loss did not improve from 0.05962
		126/126 [==============================] - 2s 14ms/step - loss: 0.0653 - acc: 0.9798
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0600 - acc: 0.9819
		Epoch 7: loss did not improve from 0.05962
		126/126 [==============================] - 2s 14ms/step - loss: 0.0601 - acc: 0.9819
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0578 - acc: 0.9826
		Epoch 8: loss improved from 0.05962 to 0.05780, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0578 - acc: 0.9825
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0576 - acc: 0.9823
		Epoch 9: loss improved from 0.05780 to 0.05756, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0576 - acc: 0.9823
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0580 - acc: 0.9828
		Epoch 10: loss did not improve from 0.05756
		126/126 [==============================] - 2s 14ms/step - loss: 0.0580 - acc: 0.9828
		Q: 날씨 죽인다
		A: 보일러 를 켜세요 
		
		
		Q: 냉면 땡긴다
		A: 생각 만 해도 군침 이 도 네 요 
		
		
		Q: 겨울 지나 봄 이야
		A: 마음 감기 조심하세요 
		
		
		processing epoch: 101...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0559 - acc: 0.9831
		Epoch 1: loss improved from 0.05756 to 0.05596, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0560 - acc: 0.9832
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0562 - acc: 0.9830
		Epoch 2: loss did not improve from 0.05596
		126/126 [==============================] - 2s 14ms/step - loss: 0.0562 - acc: 0.9830
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0559 - acc: 0.9832
		Epoch 3: loss did not improve from 0.05596
		126/126 [==============================] - 2s 14ms/step - loss: 0.0560 - acc: 0.9832
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0567 - acc: 0.9830
		Epoch 4: loss did not improve from 0.05596
		126/126 [==============================] - 2s 14ms/step - loss: 0.0567 - acc: 0.9830
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0538 - acc: 0.9839
		Epoch 5: loss improved from 0.05596 to 0.05384, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0538 - acc: 0.9839
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0551 - acc: 0.9828
		Epoch 6: loss did not improve from 0.05384
		126/126 [==============================] - 2s 14ms/step - loss: 0.0551 - acc: 0.9829
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0551 - acc: 0.9835
		Epoch 7: loss did not improve from 0.05384
		126/126 [==============================] - 2s 14ms/step - loss: 0.0551 - acc: 0.9835
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0551 - acc: 0.9841
		Epoch 8: loss did not improve from 0.05384
		126/126 [==============================] - 2s 14ms/step - loss: 0.0550 - acc: 0.9841
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0512 - acc: 0.9846
		Epoch 9: loss improved from 0.05384 to 0.05128, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0513 - acc: 0.9846
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0518 - acc: 0.9846
		Epoch 10: loss did not improve from 0.05128
		126/126 [==============================] - 2s 14ms/step - loss: 0.0518 - acc: 0.9846
		Q: 건강 이 최고 인 것 같아
		A: 가장 중요한 목표 네 요 
		
		
		Q: 넘어질 뻔했어
		A: 다치지 않으셨나 걱정 이네 요 
		
		
		Q: 누가 만들었니
		A: 마음 과 마음 을 이 어보 고 싶어하는 사람 이 만들었어요 
		
		
		processing epoch: 111...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0519 - acc: 0.9846
		Epoch 1: loss did not improve from 0.05128
		126/126 [==============================] - 2s 15ms/step - loss: 0.0520 - acc: 0.9845
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0508 - acc: 0.9847
		Epoch 2: loss improved from 0.05128 to 0.05080, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0508 - acc: 0.9847
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0500 - acc: 0.9851
		Epoch 3: loss improved from 0.05080 to 0.04996, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0500 - acc: 0.9851
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0520 - acc: 0.9845
		Epoch 4: loss did not improve from 0.04996
		126/126 [==============================] - 2s 15ms/step - loss: 0.0521 - acc: 0.9845
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0490 - acc: 0.9855
		Epoch 5: loss improved from 0.04996 to 0.04904, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0490 - acc: 0.9855
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0498 - acc: 0.9854
		Epoch 6: loss did not improve from 0.04904
		126/126 [==============================] - 2s 14ms/step - loss: 0.0498 - acc: 0.9854
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0471 - acc: 0.9859
		Epoch 7: loss improved from 0.04904 to 0.04708, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0471 - acc: 0.9860
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0457 - acc: 0.9867
		Epoch 8: loss improved from 0.04708 to 0.04567, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0457 - acc: 0.9867
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.0455 - acc: 0.9868
		Epoch 9: loss improved from 0.04567 to 0.04550, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0455 - acc: 0.9868
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0454 - acc: 0.9869
		Epoch 10: loss did not improve from 0.04550
		126/126 [==============================] - 2s 17ms/step - loss: 0.0456 - acc: 0.9868
		Q: 내 잘못 이 뭔 지 모르겠어
		A: 모르는 게 잘못 인 거 같아요 
		
		
		Q: 기숙사 떨어졌어
		A: 다음 학기 에는 학점 관리 를 더 열심히 해봐요 
		
		
		Q: 나도 커플룩 입고 싶다
		A: 커플 부터 만드세요 
		
		
		processing epoch: 121...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0437 - acc: 0.9870
		Epoch 1: loss improved from 0.04550 to 0.04373, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.0437 - acc: 0.9871
		Epoch 2/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0445 - acc: 0.9869
		Epoch 2: loss did not improve from 0.04373
		126/126 [==============================] - 2s 14ms/step - loss: 0.0445 - acc: 0.9868
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0450 - acc: 0.9872
		Epoch 3: loss did not improve from 0.04373
		126/126 [==============================] - 2s 14ms/step - loss: 0.0450 - acc: 0.9872
		Epoch 4/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0443 - acc: 0.9864
		Epoch 4: loss did not improve from 0.04373
		126/126 [==============================] - 2s 15ms/step - loss: 0.0443 - acc: 0.9864
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0429 - acc: 0.9884
		Epoch 5: loss improved from 0.04373 to 0.04283, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0428 - acc: 0.9884
		Epoch 6/10
		126/126 [==============================] - ETA: 0s - loss: 0.0439 - acc: 0.9869
		Epoch 6: loss did not improve from 0.04283
		126/126 [==============================] - 2s 14ms/step - loss: 0.0439 - acc: 0.9869
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0431 - acc: 0.9878
		Epoch 7: loss did not improve from 0.04283
		126/126 [==============================] - 2s 15ms/step - loss: 0.0432 - acc: 0.9878
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0421 - acc: 0.9881
		Epoch 8: loss improved from 0.04283 to 0.04223, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0422 - acc: 0.9880
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.0423 - acc: 0.9875
		Epoch 9: loss did not improve from 0.04223
		126/126 [==============================] - 2s 14ms/step - loss: 0.0423 - acc: 0.9875
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0475 - acc: 0.9866
		Epoch 10: loss did not improve from 0.04223
		126/126 [==============================] - 2s 14ms/step - loss: 0.0475 - acc: 0.9865
		Q: 데이트 로 놀이 공원 가는거 어때
		A: 데이트 하기 좋은 곳 이 죠 
		
		
		Q: 문신 하고 싶은데 후회 할까
		A: 나중 에 후회 할 수도 있을 것 같아요 
		
		
		Q: 나 오늘 따라 잘생겨 보이 네
		A: 자신 에게 콩깍지 가 씌였나봐 요 
		
		
		processing epoch: 131...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.0468 - acc: 0.9868
		Epoch 1: loss did not improve from 0.04223
		126/126 [==============================] - 2s 17ms/step - loss: 0.0468 - acc: 0.9868
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0493 - acc: 0.9858
		Epoch 2: loss did not improve from 0.04223
		126/126 [==============================] - 2s 14ms/step - loss: 0.0492 - acc: 0.9859
		Epoch 3/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0425 - acc: 0.9882
		Epoch 3: loss did not improve from 0.04223
		126/126 [==============================] - 2s 14ms/step - loss: 0.0425 - acc: 0.9881
		Epoch 4/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0418 - acc: 0.9881
		Epoch 4: loss improved from 0.04223 to 0.04183, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0418 - acc: 0.9881
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0427 - acc: 0.9874
		Epoch 5: loss did not improve from 0.04183
		126/126 [==============================] - 2s 14ms/step - loss: 0.0427 - acc: 0.9874
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0382 - acc: 0.9894
		Epoch 6: loss improved from 0.04183 to 0.03820, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0382 - acc: 0.9895
		Epoch 7/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0376 - acc: 0.9892
		Epoch 7: loss improved from 0.03820 to 0.03752, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0375 - acc: 0.9893
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.0379 - acc: 0.9895
		Epoch 8: loss did not improve from 0.03752
		126/126 [==============================] - 2s 15ms/step - loss: 0.0379 - acc: 0.9895
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0372 - acc: 0.9894
		Epoch 9: loss improved from 0.03752 to 0.03711, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0371 - acc: 0.9895
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0361 - acc: 0.9898
		Epoch 10: loss improved from 0.03711 to 0.03618, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.0362 - acc: 0.9897
		Q: 계속 속이 진짜 안 좋아
		A: 계속 좋지 않으면 병원 에 가보세요 
		
		
		Q: 결혼식 이 너무 많아
		A: 인맥 이 넓으신 가봐요 
		
		
		Q: 나 수학여행 간다
		A: 친구 들 과 좋은 추억 만들고 오세요 
		
		
		processing epoch: 141...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0345 - acc: 0.9906
		Epoch 1: loss improved from 0.03618 to 0.03445, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.0344 - acc: 0.9906
		Epoch 2/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0372 - acc: 0.9897
		Epoch 2: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0371 - acc: 0.9897
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0375 - acc: 0.9894
		Epoch 3: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0375 - acc: 0.9895
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0361 - acc: 0.9897
		Epoch 4: loss did not improve from 0.03445
		126/126 [==============================] - 2s 14ms/step - loss: 0.0361 - acc: 0.9897
		Epoch 5/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0413 - acc: 0.9876
		Epoch 5: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0412 - acc: 0.9876
		Epoch 6/10
		126/126 [==============================] - ETA: 0s - loss: 0.0383 - acc: 0.9889
		Epoch 6: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0383 - acc: 0.9889
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.0350 - acc: 0.9903
		Epoch 7: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0350 - acc: 0.9903
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0354 - acc: 0.9901
		Epoch 8: loss did not improve from 0.03445
		126/126 [==============================] - 2s 14ms/step - loss: 0.0354 - acc: 0.9901
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.0359 - acc: 0.9902
		Epoch 9: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0359 - acc: 0.9902
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0372 - acc: 0.9896
		Epoch 10: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0371 - acc: 0.9896
		Q: 개강 옷 예쁘게 입어 볼까
		A: 개시 해보세요 
		
		
		Q: 뭐 볼까
		A: 오늘 은 예능 이 요 
		
		
		Q: 미용실 가서 머리 해야지
		A: 예쁘게 변신 하고 오세요 
		
		
		processing epoch: 151...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.0353 - acc: 0.9897
		Epoch 1: loss did not improve from 0.03445
		126/126 [==============================] - 2s 15ms/step - loss: 0.0353 - acc: 0.9897
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0337 - acc: 0.9912
		Epoch 2: loss improved from 0.03445 to 0.03372, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 17ms/step - loss: 0.0337 - acc: 0.9912
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0333 - acc: 0.9906
		Epoch 3: loss improved from 0.03372 to 0.03338, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0334 - acc: 0.9906
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.0317 - acc: 0.9916
		Epoch 4: loss improved from 0.03338 to 0.03169, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0317 - acc: 0.9916
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0304 - acc: 0.9916
		Epoch 5: loss improved from 0.03169 to 0.03048, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0305 - acc: 0.9916
		Epoch 6/10
		126/126 [==============================] - ETA: 0s - loss: 0.0301 - acc: 0.9919
		Epoch 6: loss improved from 0.03048 to 0.03012, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0301 - acc: 0.9919
		Epoch 7/10
		126/126 [==============================] - ETA: 0s - loss: 0.0316 - acc: 0.9913
		Epoch 7: loss did not improve from 0.03012
		126/126 [==============================] - 2s 15ms/step - loss: 0.0316 - acc: 0.9913
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0330 - acc: 0.9914
		Epoch 8: loss did not improve from 0.03012
		126/126 [==============================] - 2s 14ms/step - loss: 0.0329 - acc: 0.9915
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0305 - acc: 0.9917
		Epoch 9: loss did not improve from 0.03012
		126/126 [==============================] - 2s 15ms/step - loss: 0.0305 - acc: 0.9918
		Epoch 10/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0316 - acc: 0.9912
		Epoch 10: loss did not improve from 0.03012
		126/126 [==============================] - 2s 14ms/step - loss: 0.0317 - acc: 0.9912
		Q: 나 한테 만 왜 이런 일이 일어날까
		A: 다른 사람 도 그럴 거 예요 
		
		
		Q: 네일 할까
		A: 기분 전환 을 해보세요 
		
		
		Q: 드디어 천일
		A: 축하 합니다 
		
		
		processing epoch: 161...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.0317 - acc: 0.9912
		Epoch 1: loss did not improve from 0.03012
		126/126 [==============================] - 2s 15ms/step - loss: 0.0317 - acc: 0.9912
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.0305 - acc: 0.9918
		Epoch 2: loss did not improve from 0.03012
		126/126 [==============================] - 2s 15ms/step - loss: 0.0305 - acc: 0.9918
		Epoch 3/10
		126/126 [==============================] - ETA: 0s - loss: 0.0290 - acc: 0.9923
		Epoch 3: loss improved from 0.03012 to 0.02898, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 3s 20ms/step - loss: 0.0290 - acc: 0.9923
		Epoch 4/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0277 - acc: 0.9927
		Epoch 4: loss improved from 0.02898 to 0.02781, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 18ms/step - loss: 0.0278 - acc: 0.9927
		Epoch 5/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0292 - acc: 0.9922
		Epoch 5: loss did not improve from 0.02781
		126/126 [==============================] - 2s 15ms/step - loss: 0.0292 - acc: 0.9922
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0287 - acc: 0.9921
		Epoch 6: loss did not improve from 0.02781
		126/126 [==============================] - 2s 15ms/step - loss: 0.0287 - acc: 0.9921
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0309 - acc: 0.9912
		Epoch 7: loss did not improve from 0.02781
		126/126 [==============================] - 2s 15ms/step - loss: 0.0309 - acc: 0.9912
		Epoch 8/10
		126/126 [==============================] - ETA: 0s - loss: 0.0305 - acc: 0.9916
		Epoch 8: loss did not improve from 0.02781
		126/126 [==============================] - 2s 15ms/step - loss: 0.0305 - acc: 0.9916
		Epoch 9/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0276 - acc: 0.9928
		Epoch 9: loss improved from 0.02781 to 0.02769, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.0277 - acc: 0.9928
		Epoch 10/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0272 - acc: 0.9927
		Epoch 10: loss improved from 0.02769 to 0.02743, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0274 - acc: 0.9927
		Q: 뭐 하는 중
		A: 독서 와 음악 감상 이라고 하고 싶지만 아무 것 도 안 했어요 
		
		
		Q: 눈썹 문신 해볼까
		A: 깔끔해질 거 예요 
		
		
		Q: 긴장 돼
		A: 크게 숨 한 번 쉬어 보세요 
		
		
		processing epoch: 171...
		Epoch 1/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0277 - acc: 0.9926
		Epoch 1: loss did not improve from 0.02743
		126/126 [==============================] - 2s 17ms/step - loss: 0.0277 - acc: 0.9926
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.0268 - acc: 0.9927
		Epoch 2: loss improved from 0.02743 to 0.02677, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0268 - acc: 0.9927
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0256 - acc: 0.9931
		Epoch 3: loss improved from 0.02677 to 0.02567, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0257 - acc: 0.9931
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.0251 - acc: 0.9932
		Epoch 4: loss improved from 0.02567 to 0.02515, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0251 - acc: 0.9932
		Epoch 5/10
		126/126 [==============================] - ETA: 0s - loss: 0.0284 - acc: 0.9930
		Epoch 5: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0284 - acc: 0.9930
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0317 - acc: 0.9916
		Epoch 6: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0316 - acc: 0.9917
		Epoch 7/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0391 - acc: 0.9912
		Epoch 7: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0390 - acc: 0.9913
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0418 - acc: 0.9900
		Epoch 8: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0418 - acc: 0.9900
		Epoch 9/10
		126/126 [==============================] - ETA: 0s - loss: 0.0328 - acc: 0.9922
		Epoch 9: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0328 - acc: 0.9922
		Epoch 10/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0281 - acc: 0.9932
		Epoch 10: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0281 - acc: 0.9932
		Q: 내 가 쉬워 보이 나
		A: 그렇게 대우 하는 사람 만나지 마 요 
		
		
		Q: 가끔 뭐 하는지 궁금해
		A: 그 사람 도 그럴 거 예요 
		
		
		Q: 늦은 시간 에 걸어다니니까 무서워
		A: 안전 귀가 하세요 
		
		
		processing epoch: 181...
		Epoch 1/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0279 - acc: 0.9926
		Epoch 1: loss did not improve from 0.02515
		126/126 [==============================] - 2s 17ms/step - loss: 0.0279 - acc: 0.9926
		Epoch 2/10
		126/126 [==============================] - ETA: 0s - loss: 0.0281 - acc: 0.9929
		Epoch 2: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0281 - acc: 0.9929
		Epoch 3/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0266 - acc: 0.9937
		Epoch 3: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0266 - acc: 0.9936
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.0258 - acc: 0.9943
		Epoch 4: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0258 - acc: 0.9943
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0259 - acc: 0.9935
		Epoch 5: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0264 - acc: 0.9935
		Epoch 6/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0261 - acc: 0.9934
		Epoch 6: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0260 - acc: 0.9934
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0282 - acc: 0.9925
		Epoch 7: loss did not improve from 0.02515
		126/126 [==============================] - 2s 15ms/step - loss: 0.0281 - acc: 0.9925
		Epoch 8/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0238 - acc: 0.9942
		Epoch 8: loss improved from 0.02515 to 0.02378, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.0238 - acc: 0.9942
		Epoch 9/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0232 - acc: 0.9940
		Epoch 9: loss improved from 0.02378 to 0.02326, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.0233 - acc: 0.9940
		Epoch 10/10
		126/126 [==============================] - ETA: 0s - loss: 0.0222 - acc: 0.9948
		Epoch 10: loss improved from 0.02326 to 0.02224, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.0222 - acc: 0.9948
		Q: 금수 저 물 고 태어나면 좋겠지
		A: 뭔가 안 풀리는 일이 있나 봐요 
		
		
		Q: 내 가 만족 을 못 해
		A: 스스로 좋다고 못 느끼는게 제일 어려운 것 같아요 
		
		
		Q: 맛집 다녀왔어
		A: 맛있게 드시고 오셨길 바랍니다 
		
		
		processing epoch: 191...
		Epoch 1/10
		126/126 [==============================] - ETA: 0s - loss: 0.0218 - acc: 0.9946
		Epoch 1: loss improved from 0.02224 to 0.02178, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 19ms/step - loss: 0.0218 - acc: 0.9946
		Epoch 2/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0234 - acc: 0.9943
		Epoch 2: loss did not improve from 0.02178
		126/126 [==============================] - 2s 15ms/step - loss: 0.0234 - acc: 0.9942
		Epoch 3/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0247 - acc: 0.9935
		Epoch 3: loss did not improve from 0.02178
		126/126 [==============================] - 2s 15ms/step - loss: 0.0246 - acc: 0.9935
		Epoch 4/10
		126/126 [==============================] - ETA: 0s - loss: 0.0219 - acc: 0.9949
		Epoch 4: loss did not improve from 0.02178
		126/126 [==============================] - 2s 15ms/step - loss: 0.0219 - acc: 0.9949
		Epoch 5/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0212 - acc: 0.9948
		Epoch 5: loss improved from 0.02178 to 0.02129, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 15ms/step - loss: 0.0213 - acc: 0.9948
		Epoch 6/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0226 - acc: 0.9941
		Epoch 6: loss did not improve from 0.02129
		126/126 [==============================] - 2s 15ms/step - loss: 0.0226 - acc: 0.9941
		Epoch 7/10
		124/126 [============================>.] - ETA: 0s - loss: 0.0217 - acc: 0.9949
		Epoch 7: loss did not improve from 0.02129
		126/126 [==============================] - 2s 15ms/step - loss: 0.0219 - acc: 0.9949
		Epoch 8/10
		123/126 [============================>.] - ETA: 0s - loss: 0.0195 - acc: 0.9953
		Epoch 8: loss improved from 0.02129 to 0.01947, saving model to model/seq2seq-attention-checkpoint.ckpt
		126/126 [==============================] - 2s 16ms/step - loss: 0.0195 - acc: 0.9953
		Epoch 9/10
		125/126 [============================>.] - ETA: 0s - loss: 0.0197 - acc: 0.9953
		Epoch 9: loss did not improve from 0.01947
		126/126 [==============================] - 2s 15ms/step - loss: 0.0197 - acc: 0.9953
		Epoch 10/10
		126/126 [==============================] - ETA: 0s - loss: 0.0218 - acc: 0.9945
		Epoch 10: loss did not improve from 0.01947
		126/126 [==============================] - 2s 15ms/step - loss: 0.0218 - acc: 0.9945
		Q: 내 가 좋아하는 사람 이 나 안 좋아하는 거 같아
		A: 슬픈 이야기 네 요 
		
		
		Q: 내일 하루 종일 바빠
		A: 바빠도 힘내세요 
		
		
		Q: 괜히 농담 해서 망했다
		A: 늦지 않았어요

Code cell <eeaEokUFeP_v>
#%% [code]
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))
Execution output from Aug 10, 2022 8:15 AM
1KB
	Stream
		<< 말을 걸어 보세요!
		챗봇
		>> 챗봇 응답: 거창하지 않아도 돼요 
		<< 말을 걸어 보세요!
		오늘 하루 좋은 일만 가득했으면 좋겠어.
		>> 챗봇 응답: 친구 와 수다 를 떠세요 
		<< 말을 걸어 보세요!
		오늘 하루 좋은 일만 가득하길..
		>> 챗봇 응답: 마음 과 마음 을 이 어보 고 싶어하는 사람 이 만들었어요 
		<< 말을 걸어 보세요!
		좋은 일만 가득하길.
		>> 챗봇 응답: 저기압 에는 고 기 앞 이 죠 
		<< 말을 걸어 보세요!
		좋은 일만 가득하길.....
		>> 챗봇 응답: 저기압 에는 고 기 앞 이 죠 
		<< 말을 걸어 보세요!
		오늘 하루 좋은 일만..
		>> 챗봇 응답: 외로우니까 사람 이다 
		<< 말을 걸어 보세요!
		오늘 하루 좋은 일만.
		>> 챗봇 응답: 외로우니까 사람 이다 
		<< 말을 걸어 보세요!
		q

Code cell <WwFVuD7ueTPB>
#%% [code]


Notebook attributes
Collapse sections ["4TPsL7c-nKLm","PRF7_GT9nKLo","y2SKSCVTnKLr","RLR7e81DnKLt","b3WJLF-pnKLx"]
